In [1]:
"""


Plott DPR VCUT etc...


"""

"""
Analyse of GPM DPR BB

"""




'\nAnalyse of GPM DPR BB\n\n'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wradlib
import wradlib as wrl
from osgeo import osr
import h5py
import glob
from pcc import get_my_cmap
from pcc import get_miub_cmap
from pcc import boxpol_pos
bonn_pos = boxpol_pos()
bx, by = bonn_pos['gkx_ppi'], bonn_pos['gky_ppi']
bonnlat, bonnlon = bonn_pos['lat_ppi'], bonn_pos['lon_ppi']
blat, blon = bonn_pos['lat_ppi'], bonn_pos['lon_ppi']


from pcc import plot_borders
from pcc import plot_radar

#% matplotlib inline

print ("  ")

In [3]:
zt = '20141007'; pfadnr = 0
scan  = 'NS'#NS' #or MS

def read_dpr(zt, scan, pfadnr):

    pfad = ('/automount/ags/velibor/gpmdata/dpr/2A.GPM.DPR.V6-20160118.'+zt+'*.HDF5')
    dpr_pfad = sorted(glob.glob(pfad))[pfadnr]

    print dpr_pfad

    # Einlesen
    dpr = h5py.File(dpr_pfad, 'r')
    # Position
    dpr_lat=np.array(dpr[scan]['Latitude'])
    dpr_lon=np.array(dpr[scan]['Longitude'])

    # Refelctivity Volumen
    dpr_pp=np.array(dpr[scan]['SLV']['zFactorCorrected'])
    dpr_pp[dpr_pp<0]= np.nan
    print dpr_pp.shape

    #Projection
    proj_stereo = wrl.georef.create_osr("dwd-radolan")
    proj_wgs = osr.SpatialReference()
    proj_wgs.ImportFromEPSG(4326)

    from pcc import boxpol_pos
    bonn_pos = boxpol_pos()
    bx, by = bonn_pos['gkx_ppi'], bonn_pos['gky_ppi']
    bonnlat, bonnlon = bonn_pos['lat_ppi'], bonn_pos['lon_ppi']
    blat, blon = bonn_pos['lat_ppi'], bonn_pos['lon_ppi']

    dpr_lon, dpr_lat = wradlib.georef.reproject(dpr_lon, dpr_lat, projection_target=proj_stereo , projection_source=proj_wgs)
    bonnlon, bonnlat = wradlib.georef.reproject(bonnlon, bonnlat, projection_target=proj_stereo , projection_source=proj_wgs)

    # BoXPol region
    lon0, lat0, radius = bonnlon, bonnlat, 100
    r = np.sqrt((dpr_lat - lat0)**2 + (dpr_lon - lon0)**2)
    position = r < radius

    
    dpr_pp[np.where(r > radius)]=np.nan
    
    lat = dpr_lat[position]
    lon = dpr_lon[position]
    dpr_pp = dpr_pp[position]



    # Height of dpr
    h = np.arange(150,4800,150)
    if scan=='HS':
        hdpr = 1000 * (np.arange(88,0,-1)*0.250)

    else:
        hdpr = 1000 * (np.arange(176,0,-1)*0.125)

    
    return lat, lon, hdpr, dpr_pp

In [4]:
lat, lon, alt,  dpr_z = read_dpr(zt, scan, pfadnr)

/automount/ags/velibor/gpmdata/dpr/2A.GPM.DPR.V6-20160118.20141007-S015721-E032951.003445.V04A.HDF5
(7930, 49, 176)


In [5]:
print (lat.shape, lon.shape, dpr_z.shape, alt.shape)

((1129,), (1129,), (1129, 176), (176,))


In [39]:
plt.scatter(lat,lon, c=np.ma.masked_invalid(dpr_z[:,160]))
plt.colorbar()
plt.show()

In [43]:
plt.imshow(np.ma.masked_invalid(dpr_z).transpose())
plt.colorbar()
plt.show()
#plt.subplot(2,2,1)
#plt.pcolormesh(lon,lat,np.ma.masked_invalid(dpr_z[:,160]))
#plt.colorbar()
#plt.xlim(-350,-100)
#plt.ylim(-4350, -4100)#

#plt.subplot(2,2,2)
#plt.pcolormesh(lon[:,20],alt,np.ma.masked_invalid(dpr_z[:,20,:].transpose()))
#plt.colorbar()
#plt.ylim(0,22000)
#plt.xlim(-350,-100)

#plt.show()


In [32]:

xs, ys, zs, co = np.array([]),np.array([]),np.array([]),np.array([])



for z in range(0,176,10):
    for x in range(1129):
            xs = np.append(xs,lon[x])
            ys = np.append(ys,lat[x])
            zs = np.append(zs,alt[z])
            co = np.append(co,dpr_z[x,z])

            


In [22]:
plt.scatter(xs,ys,c=co)
plt.show()

In [33]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
pl = ax.scatter(xs, ys, zs, c=co)
fig.colorbar(pl, shrink=0.5, aspect=5)
plt.show()


In [20]:
176./2/2


44.0

In [ ]:


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# For each set of style and range settings, plot n random points in the box
# defined by x in [23, 32], y in [0, 100], z in [zlow, zhigh].
for z in range(156,160,2):
    for x in range(1000):
        for y in range(10):
            xs = lon[x,y]#lon.reshape(lon.shape[0]*lon.shape[1])
            ys = lat[x,y]#lat.reshape(lat.shape[0]*lat.shape[1])
            zs = alt[z]
            co = dpr_z[x,y,z]
            #print co
            ax.scatter(xs, ys, zs, c=co)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()


In [ ]:
xs = lon.reshape(lon.shape[0]*lon.shape[1])
ys = lat.reshape(lat.shape[0]*lat.shape[1])
zs = lat.reshape(lat.shape[0]*lat.shape[1])
ds = dpr_z.reshape(dpr_z.shape[0]*dpr_z.shape[1]*dpr_z.shape[2])

In [ ]:
xs.shape, ys.shape